# Movielens 영화 추천

In [1]:
# 필요한 모듈 import
import os
import pandas as pd
import numpy as np

import scipy
from scipy.sparse import csr_matrix

import implicit
from implicit.als import AlternatingLeastSquares

## Step 1. 데이터 준비와 전처리

In [2]:
# Movielens 데이터 불러오기
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# ratings 3점 이상만 남기기
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꾸기
ratings.rename(columns={'ratings':'counts'}, inplace=True)
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어오기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 데이터프레임 합치기
movie_data = pd.merge(ratings, movies)
movie_data

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


## Step 2. 분석하기

In [7]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
# 가장 인기 있는 영화 30개(인기순) - 인기도는 많이 본 영화를 기준
famous_movie = movie_data.groupby('title')['user_id'].count()
famous_movie.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [10]:
# 유저별 몇 개의 영화를 봤는지에 대한 통계
user_count = movie_data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

## Step 3. 선호하는 영화를 5가지 골라서 ratings에 추가하기

In [11]:
# 선호하는 영화 5가지
my_favorite = ['Titanic (1997)' , 'Toy Story (1995)' ,'Forrest Gump (1994)' ,'Sound of Music, The (1965)' ,'Lion King, The (1994)']
my_favorite_id = [1721, 1, 356, 1035, 364]

# 'soy'라는 user_id가 위 영화 counts에 5점을 부여했다고 가정
my_movie = pd.DataFrame({'user_id': ['soy']*5, 'movie_id': my_favorite_id, 'counts':[5]*5})

if not ratings.isin({'user_id':['soy']})['user_id'].any():
    ratings = ratings.append(my_movie)
# user_id에 'soy'이라는 데이터가 없다면 위에 임의로 만든 my_movie 데이터를 추가
    
ratings.tail(10)

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,soy,1721,5,NaN
1,soy,1,5,NaN
2,soy,356,5,NaN
3,soy,1035,5,NaN
4,soy,364,5,NaN


In [12]:
# 데이터프레임 합치기
movie_data = pd.merge(ratings, movies)
movie_data

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,9.783008e+08,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,9.782984e+08,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,9.782202e+08,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,9.781993e+08,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,9.781585e+08,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836478,5851,3607,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,9.583469e+08,Slaughterhouse (1987),Horror
836480,5854,690,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [13]:
# movie_data에 있는 유니크한 사용자 수
movie_data['user_id'].nunique()

6040

In [14]:
# indexing하기 (user와 movie 각각에 번호를 붙이기)

# 고유한 유저, 영화를 찾아내기
user_unique = movie_data['user_id'].unique()
movie_unique = movie_data['title'].unique()

# 유저, 영화를 indexing 
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [15]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['soy']) # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 한다. 
print(movie_to_idx['Titanic (1997)'])

4685
27


- user를 인덱싱할 때, 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 하는데 아무리 찾아봐도 어디서 잘못된 건지 알 수가 없다.

In [16]:
# indexing을 통해 데이터 컬럼 내 값 바꾸기

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구하기
# 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거
temp_user_data = movie_data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movie_data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie_data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱
temp_movie_data = movie_data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(movie_data):
    print('movie_id column indexing OK!!')
    movie_data['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

movie_data

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts,timestamp,title,genre
0,0,0,5,9.783008e+08,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,0,5,9.782984e+08,One Flew Over the Cuckoo's Nest (1975),Drama
2,2,0,4,9.782202e+08,One Flew Over the Cuckoo's Nest (1975),Drama
3,3,0,4,9.781993e+08,One Flew Over the Cuckoo's Nest (1975),Drama
4,4,0,5,9.781585e+08,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836478,1621,3623,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836479,3481,3624,4,9.583469e+08,Slaughterhouse (1987),Horror
836480,3481,3625,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836481,4159,3626,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary


- 처음에 따로 인덱싱하는 작업을 거쳤는데, movie_id를 indexing 하는 과정에서 각 영화에 맞는 id로 indexing 되어야 하는데 그냥 앞에서부터 순서대로 3628~3632까지 indexing되었다. 영화 이름을 정확하게 맞춘 상태라고 생각했는데, 뭐가 문제인지 정확하게 모르겠다.  
- 그래서 각 영화에 대한 id를 따로 가져와서 맞춰 준 상태에서 진행하였다. 다행히 이렇게 진행했을 때에는 영화인덱싱에 문제가 안생기는 거 같다. 하지만, 유저 인덱싱에서도 문제가 생겼는데, 해결이 안된다. 

## Step 4. CSR matrix 만들어보기

In [17]:
num_user = movie_data['user_id'].nunique()
num_movie = movie_data['title'].nunique()

csr_data = csr_matrix((movie_data.counts, (movie_data.user_id, movie_data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## Step 5. als_model = AlternatingLeastSquares 모델 구성 후 훈련하기

In [32]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose하기
csr_data_transpose = csr_data.T

In [33]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

## Step 6. 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 선호도 파악하기

In [34]:
soy, titanic = user_to_idx['soy'], movie_to_idx['Titanic (1997)']
soy_vector, titanic_vector = als_model.user_factors[soy], als_model.item_factors[titanic]

In [35]:
soy_vector

array([ 0.8209238 , -0.8332656 ,  0.25572735, -0.06132647, -0.0039618 ,
       -0.02145077, -0.09180992, -0.26908422, -0.35818946, -0.11887038,
       -0.01605035, -0.59763294, -0.07801373, -0.7110644 , -0.19682933,
       -0.66265666,  0.02868234, -0.26140535,  0.3833715 ,  0.45510477,
        0.02558705, -0.1657889 ,  0.2995533 , -0.09670059,  0.25739473,
       -0.17070527, -0.27573165,  0.35770512, -0.00619934, -0.12263353,
        0.27534524,  0.53387475, -0.23652685, -0.07365609,  0.3373982 ,
        0.26986298, -0.55690277,  0.11179329,  0.3350459 , -0.28602564,
       -0.6939901 , -0.07791303, -0.53488785,  0.63089186,  0.09937033,
        0.079178  , -0.20819116, -0.07595681,  0.67215896,  0.00487602,
        0.2905618 ,  0.5825785 ,  0.348188  ,  0.33952567, -0.9236209 ,
        0.20671979,  0.2750603 ,  0.80110544, -0.39369622, -0.02569204,
       -0.17877398,  0.3396373 , -0.18215019,  0.1839023 ,  0.4542532 ,
        0.1860256 , -0.14941935, -0.36622941,  0.33456305, -0.01

In [36]:
titanic_vector

array([ 5.87751269e-02, -1.82476193e-02,  1.60714239e-02, -3.07025816e-02,
       -3.49096134e-02,  8.67093261e-03,  4.90531437e-02, -5.85547090e-03,
       -3.25533301e-02, -6.76504010e-03,  2.15380378e-02, -1.41401831e-02,
        2.09118798e-02, -1.25331925e-02,  2.53722700e-03, -5.58337988e-03,
        1.44888703e-02,  1.18770590e-02,  5.55218430e-03,  2.74694357e-02,
        1.70185417e-02,  8.86500720e-03,  1.31984835e-03, -3.93474853e-04,
        1.83681231e-02, -2.47154757e-02, -4.78612864e-03,  1.22609632e-02,
       -1.43110631e-02,  5.79913100e-03,  2.49188039e-02,  6.69897115e-03,
       -2.60349293e-03, -4.01035650e-03, -5.62458625e-03,  3.84300724e-02,
       -2.31462047e-02, -8.36731121e-03,  3.76196094e-02,  1.64223593e-02,
       -2.75474950e-03,  9.34037566e-03, -2.83457600e-02,  2.72773281e-02,
       -2.28293296e-02,  7.44854053e-03, -1.75925475e-02, -8.17127619e-03,
       -1.68598033e-02,  1.35601154e-02,  2.37027034e-02,  4.64087911e-03,
        1.20109878e-02,  

In [37]:
# soy와 titanic을 내적
np.dot(soy_vector, titanic_vector)

0.80271536

- 1에 가깝게 나올수록 학습이 잘 되었다고 볼 수 있다. 너무 작은 경우, factors를 늘리거나 iterations를 늘려야 한다.  
- factors=100, iterations=15일 때, 내적 값은 0.6412096이 나왔다.  
- factors=200, iterations=20일 때, 내적 값은 0.80271536이 나왔다. 이를 통해 factors와 iterations를 적당히 늘리면 학습이 더 잘 될 수 있다는 것을 직접 확인해 보았다.

In [38]:
# soy와 Forrest Gump (1994)를 내적
forrest_gump = movie_to_idx['Forrest Gump (1994)']
forrest_gump_vector = als_model.item_factors[forrest_gump]
np.dot(soy_vector, forrest_gump_vector)

0.82279605

## Step 7. 좋아하는 영화와 비슷한 영화 추천받기

In [39]:
# similar_items 메서드를 통하여 titanic과 비슷한 영화 찾기
favorite_movie = 'Titanic (1997)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(27, 1.0000001),
 (1668, 0.31456566),
 (498, 0.31430653),
 (1638, 0.31292716),
 (626, 0.304258),
 (385, 0.29733407),
 (3326, 0.2947154),
 (2895, 0.29191893),
 (3399, 0.2912452),
 (3051, 0.2898655),
 (3376, 0.28787398),
 (508, 0.2865569),
 (2798, 0.28512615),
 (3257, 0.28509212),
 (2957, 0.28485456)]

In [41]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict 생성
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Titanic (1997)',
 'Snow Day (2000)',
 'Ever After: A Cinderella Story (1998)',
 'Walking Dead, The (1995)',
 "You've Got Mail (1998)",
 'Truman Show, The (1998)',
 'Train of Life (Train De Vie) (1998)',
 'American Pop (1981)',
 'Panther (1995)',
 'Hot Spot, The (1990)',
 'Saragossa Manuscript, The (Rekopis znaleziony w Saragossie) (1965)',
 'Ghost (1990)',
 'Chopping Mall (a.k.a. Killbots) (1986)',
 'Paris, France (1993)',
 'Horror Express (1972)']

In [42]:
# 위의 과정이 담긴 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [44]:
# 확인(Sound of Music, The)
get_similar_movie('Sound of Music, The (1965)')

['Sound of Music, The (1965)',
 'Oliver! (1968)',
 'My Fair Lady (1964)',
 'West Side Story (1961)',
 'King and I, The (1956)',
 'Gigi (1958)',
 'Mary Poppins (1964)',
 'White Christmas (1954)',
 "Rosemary's Baby (1968)",
 'Phantom of the Opera, The (1943)']

## Step 8. 가장 좋아할 만한 영화 추천받기

In [45]:
user = user_to_idx['soy']
# recommend에서는 user*item CSR Matrix를 받는다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(33, 0.42264327),
 (10, 0.39412194),
 (50, 0.34399915),
 (45, 0.24633175),
 (30, 0.20580798),
 (508, 0.2048539),
 (360, 0.20484325),
 (255, 0.20383382),
 (322, 0.20083323),
 (276, 0.19374602),
 (34, 0.18318427),
 (8, 0.18233386),
 (1844, 0.1781485),
 (191, 0.17432834),
 (1509, 0.16984655),
 (16, 0.16856393),
 (38, 0.16660392),
 (154, 0.16461277),
 (271, 0.16380867),
 (2, 0.16377375)]

In [46]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Aladdin (1992)',
 'Beauty and the Beast (1991)',
 'Toy Story 2 (1999)',
 'Mary Poppins (1964)',
 'Antz (1998)',
 'Ghost (1990)',
 'Dirty Dancing (1987)',
 "There's Something About Mary (1998)",
 'Babe (1995)',
 'North by Northwest (1959)',
 'Mulan (1998)',
 'Snow White and the Seven Dwarfs (1937)',
 'Oliver! (1968)',
 'Little Mermaid, The (1989)',
 'English Patient, The (1996)',
 'Tarzan (1999)',
 'Sixth Sense, The (1999)',
 'As Good As It Gets (1997)',
 'Election (1999)',
 'My Fair Lady (1964)']

In [48]:
aladdin = movie_to_idx['Aladdin (1992)']
explain = als_model.explain(user, csr_data, itemid=aladdin)

In [49]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Lion King, The (1994)', 0.26831277253852553),
 ('Toy Story (1995)', 0.11787125062398916),
 ('Forrest Gump (1994)', 0.04622573079873821),
 ('Sound of Music, The (1965)', 0.019324606654951698),
 ('Titanic (1997)', -0.034441245894326955)]

- aladdin을 추천해준 것에 있어서 lion king과 toy story가 가장 크게 기여했음을 알 수 있다.

------

## 회고
1. CSR matrix가 정상적으로 만들어졌는가?  
인덱싱하는 과정에서 어려움을 겪기는 했지만 사용자와 아이템 개수를 바탕으로 CSR matrix 자체는 정확한 사이즈로 만들어졌다. 인덱싱 과정은 조언도 구해보며 다시 시도해봐야할 거 같다.  
2. MF 모델이 정상적으로 훈련되고 사용자와 아이템 벡터 내적수치가 의미있게 형성되었는가?  
1에 가까울수록 훈련이 잘 되었다고 볼 수 있는데 factor와 iterations를 조절하며 잘 훈련되도록 만들어 사용자와 아이템 벡터 내적수치를 1에 가깝도록 만들 수 있었다.  
3. 비슷한 영화 찾기와 유저에게 추천하기 과정이 정상적으로 진행되었는가?  
비슷한 영화도 잘 찾아지는 거 같았고, 유저에게 추천하기에서 대표적으로 제일 먼저 나온 aladdin 영화를 바탕으로 유사도와 기여도를 측정해보았다. 기여도에 있어서 어느 정도 예상한대로 잘 나왔다.  
4. 프로젝트를 진행하면서 느낀 점 : 이번 프로젝트에서 가장 어려웠던 점이 merge 전 데이터프레임을 써야할 지 merge 후 데이터프레임을 써야할 지와 merge 후 데이터프레임을 사용할 때 movie_id 컬럼을 사용해야할 지 title 컬럼을 사용해야할 지 구분하는 것이었다. 그 때문에 차원이 맞지 않아 에러도 많이 나고 결과가 이상해서 처음부터 잘못된 점을 찾느라 시간이 많이 소요됐다. 좀 더 예리하게 데이터를 보는 능력을 키워야 할 필요성을 느꼈다.